In [344]:
import os 
from io import StringIO
import pandas as pd
from bs4 import BeautifulSoup

In [345]:
SCORE_DIR = "data2/scores"

In [346]:
box_scores = os.listdir(SCORE_DIR)

In [347]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [348]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()

    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")] #cleans up the data we can process further
    return soup

In [349]:
def read_line_score(soup):
    line_score = pd.read_html(StringIO(str(soup)), attrs = {'id': 'line_score'})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    line_score = line_score[["team","total"]]
    return line_score

In [350]:
def read_stats(soup,team,stat):
    df = pd.read_html(StringIO(str(soup)), attrs = {"id":f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [351]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [352]:
base_cols = None
games = []
for box_score in box_scores:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = list(line_score["team"])
    
    summaries = []
    for team in teams:
        basic = read_stats(soup,team,"basic")
        advanced = read_stats(soup,team,"advanced")
    
        totals = pd.concat([basic.iloc[-1,:],advanced.iloc[-1,:]]) #iloc indexes a dataframe by position
        totals.index = totals.index.str.lower()
        
        maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
        maxes.index = maxes.index.str.lower() + "_max"
    
        summary = pd.concat([totals,maxes])
    
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep = "first")) #finds all values in box scores in the first loop amnd removes duplicates
            base_cols = [b for b in base_cols if "bpm" not in b]
    
        summary = summary[base_cols]
    
        summaries.append(summary)
    summary = pd.concat(summaries,axis=1).T #.T just rotates the table
    
    game = pd.concat([summary,line_score],axis=1)
    
    game["home"] = [0,1]
    game_opp = game.iloc[::-1].reset_index() #opponent data frame
    game_opp.columns +="_opp"
    
    full_game = pd.concat([game,game_opp],axis=1)
    
    full_game["season"] = read_season_info(soup)
    
    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")
    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)

    if len(games)%100 == 0:
        print(f"{len(games)}/{len(box_scores)}")

In [353]:
games_df = pd.concat(games, ignore_index = True)
games_df.to_csv("nba_games.csv")